In [6]:
import os
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.onnx
import onnx

from google.protobuf import json_format

# First we export a PyTorch model
filename = "model.proto"
fullpath = os.path.join("/tmp", filename)

dummy_input = Variable(torch.randn(2, 5))
model = torch.nn.Sequential(
          torch.nn.Linear(5, 4),
#           torch.nn.Dropout(p=0.9),
          torch.nn.ReLU(),
          torch.nn.Linear(4, 3),
#           torch.nn.Dropout(p=0.9),
          torch.nn.ReLU(),
          torch.nn.Linear(3, 2),
          torch.nn.Softmax(dim=1),
        )
torch.onnx.export(model, dummy_input, fullpath)
print("model saved at: %s" % fullpath)


model saved at: /tmp/model.proto


In [7]:
import syft
import syft.nn
import syft.onnx
from syft import FloatTensor

In [8]:
syft_dummy_input = FloatTensor([[1,2,3,4,5],[5,4,3,2,1]], autograd=True)
syft_model = syft.nn.Sequential([
    syft.nn.Linear(5,4),
    syft.nn.Dropout(rate=0.9),
    syft.nn.ReLU(),
    syft.nn.Linear(4, 3),
    syft.nn.Dropout(rate=0.9),
    syft.nn.ReLU(),
    syft.nn.Linear(3, 2),
    syft.nn.Softmax(dim=1)
])
syft_output = syft_model(syft_dummy_input)
print(syft_model.summary())
print(syft_output)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
linear_28 (Linear)           (None, 4)                 24
_________________________________________________________________
dropout_29 (Dropout)         (dynamic)                 0
_________________________________________________________________
relu_30 (ReLU)               (dynamic)                 0
_________________________________________________________________
linear_31 (Linear)           (None, 3)                 15
_________________________________________________________________
dropout_32 (Dropout)         (dynamic)                 0
_________________________________________________________________
relu_33 (ReLU)               (dynamic)                 0
_________________________________________________________________
linear_34 (Linear)           (None, 2)                 8
_________________________________________________________________
softm

In [9]:
# Load the PyTorch model
syft_dummy_input = FloatTensor([[1,2,3,4,5],[5,4,3,2,1]], autograd=True)
syft_model = syft.onnx.load(fullpath)

syft_model.summary()
# Execute models
pytorch_output = model(dummy_input)
syft_output = syft_model(syft_dummy_input)
print(pytorch_output)
print(syft_output)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
linear_38 (Linear)           (None, 4)                 24
_________________________________________________________________
relu_39 (ReLU)               (dynamic)                 0
_________________________________________________________________
linear_40 (Linear)           (None, 3)                 15
_________________________________________________________________
relu_41 (ReLU)               (dynamic)                 0
_________________________________________________________________
linear_42 (Linear)           (None, 2)                 8
_________________________________________________________________
softmax_43 (Softmax)         (dynamic)                 0
Total params: 47
Trainable params: 47
Non-trainable params: 0
_________________________________________________________________

Variable containing:
 0.2740  0.7260
 0.2737  0.7263
[torch.Float